In [20]:
# Load the wine data from chapter 4, and create a new model with the appropriate number of input parameters
import csv
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from collections import OrderedDict

In [21]:
wine_path="./winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [22]:
col_list = next(csv.reader(open(wine_path))) #next item from the iterator
num_row = wineq_numpy.shape[0]
num_col = wineq_numpy.shape[1]
attrs = torch.tensor(np.array(wineq_numpy[:,0:11]))
ans = torch.tensor(np.array(wineq_numpy[:,11]))
attrs, ans

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 tensor([6., 6., 6.,  ..., 6., 7., 6.]))

In [65]:
quality = {0:0,1:0,2:0, 3:0, 4:0, 5:0, 6:0,7:0,8:0,9:0,10:0}
ans -= 3
for a in ans:
    quality[a.item()]+=1
quality

{0: 20,
 1: 163,
 2: 1457,
 3: 2198,
 4: 880,
 5: 175,
 6: 5,
 7: 0,
 8: 0,
 9: 0,
 10: 0}

In [23]:
def shuffle_and_devide_set(t_u: torch, t_c:torch, ratio):
    n_samples = t_u.shape[0]
    n_val = int(ratio*n_samples)

    shuffled_indices = torch.randperm(n_samples)

    train_indices = shuffled_indices[:-n_val]
    val_indices = shuffled_indices[-n_val:]

    train_indices, val_indices
    train_t_u = t_u[train_indices]
    train_t_c = t_c [train_indices]

    val_t_u = t_u[val_indices]
    val_t_c = t_c [val_indices]

    return train_t_u, train_t_c, val_t_u, val_t_c

In [29]:
attr_t, ans_t, attr_v, ans_v = shuffle_and_devide_set(attrs, ans, 0.2)
ans_t = ans_t.unsqueeze(1);
ans_v = ans_v.unsqueeze(1);

print(attr_t.shape, attr_v.shape)
print(ans_t.shape, ans_v.shape)


torch.Size([3919, 11]) torch.Size([979, 11])
torch.Size([3919, 1]) torch.Size([979, 1])


In [42]:
def compare_out_ans(output:torch, ans:torch) :
    r_out = torch.round(output.squeeze())
    ans = ans.squeeze()
    total=0
    right=0
    length = output.shape[0]
    for i in range(0,length):
        total+=1
        if(r_out[i] == ans[i]):
            right+=1
    return right, total
        


In [57]:
def training_loop(n_epochs:int, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs+1):
        t_p_train = model(t_u_train)
        loss_train=loss_fn(t_p_train, t_c_train)

        with torch.no_grad():
            t_p_val = model(t_u_val)
            loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch <=5 or epoch % 200 == 0:
            print(f"Epoch {epoch}, Training Loss {loss_train.item():.4f}, validation loss {loss_val.item():.4f}")
            right, total = compare_out_ans(t_p_val, t_c_val)
            print(f"    score: {right} / {total} = {right/total*100:.2f}%")

In [66]:
model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(11,22)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(22,7)),
    ('output_softmax', nn.LogSoftmax(dim=1)    )
]))
optimizer = optim.Adam(model.parameters(), lr =1e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    2000, 
    optimizer, 
    model, 
    loss_fn, 
    attr_t, 
    attr_v,
    ans_t, 
    ans_v
)



RuntimeError: 0D or 1D target tensor expected, multi-target not supported

{0: 0,
 1: 0,
 2: 0,
 3: 20,
 4: 163,
 5: 1457,
 6: 2198,
 7: 880,
 8: 175,
 9: 5,
 10: 0}